In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
page = requests.get("https://www.ceara.gov.br/decretos-do-governo-do-ceara-com-acoes-contra-o-coronavirus/")

soup = BeautifulSoup(page.content, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html class="no-js" lang="pt-BR">
 <head>
  <meta charset="utf-8"/>
  <title>
   Decretos do Governo do Ceará com ações contra o coronavírus - Governo do Estado do Ceará
  </title>
  <meta content="width=device-width, initial-scale=1, user-scalable=no" id="viewport" name="viewport"/>
  <link href="https://www.ceara.gov.br/wp-content/themes/ceara2017/favicon.ico" rel="shortcut icon" type="image/x-png"/>
  <link href="https://www.ceara.gov.br/wp-content/themes/ceara2017/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="https://www.ceara.gov.br/wp-content/themes/ceara2017/favicon.ico" rel="icon" type="image/x-icon"/>
  <!-- This site is optimized with the Yoast SEO plugin v12.4 - https://yoast.com/wordpress/plugins/seo/ -->
  <meta content="max-snippet:-1, max-image-preview:large, max-video-preview:-1" name="robots">
   <link href="https://www.ceara.gov.br/decretos-do-governo-do-ceara-com-acoes-contra-o-coronavirus/" rel="canonical">
    <meta content="p

In [3]:
decree_list = soup.find("ul", {"class": "ListaEst"})

In [4]:
decrees = []
for link in decree_list.find_all('a'):
    decrees.append(link.text)

In [5]:
df_decrees = pd.DataFrame(data={'decree': decrees})

In [7]:
df_decrees['decree_id'] = df_decrees.decree.agg(lambda x: re.findall(r'[0-9]+',x)[0] + re.findall(r'[0-9]+', x)[1])

In [8]:
df_decrees['decree_day'] = df_decrees.decree.agg(lambda x: re.findall(r'[0-9]+',x)[2])
df_decrees['decree_year'] = df_decrees.decree.agg(lambda x: re.findall(r'[0-9]+', x)[3])

In [9]:
df_months_aux = pd.DataFrame(data={'month_name': ['janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho',
                                'agosto', 'setembro', 'outubro','novembro', 'dezembro'],
                 'month_id': list(range(1,13))})

In [10]:
df_decrees['decree_month_name'] = df_decrees.decree.agg(lambda x: x.split('de')[2].strip()).replace('','dezembro')

In [11]:
df_decrees = df_decrees.join(df_months_aux.set_index('month_name'), on='decree_month_name')

In [12]:
df_decrees = df_decrees.drop(columns=['decree'])

In [13]:
df_decrees['decree_date'] = df_decrees[['decree_year', 'month_id', 'decree_day']].astype(str).agg('-'.join, axis=1)

In [15]:
df_decrees.to_csv('decrees.csv', index=False)